In [1]:
import yt
import ytree
import numpy as np
import matplotlib.pyplot as plt
import json


In [2]:
yt.enable_plugins()

yt : [INFO     ] 2023-01-31 11:41:58,812 Loading plugins from /home/dskinner6/.config/yt/my_plugins.py


In [12]:
run_dir = '../'

stars = ['p3_living', 'p3_binary', 'ns_binary', 'bh', 'p2', 'dm']
types = [5, 12, 13, 6, 7, 1]

with open('param_info.json') as f:
    param_info = json.load(f)
    
with open('run_DD_data.json') as f:
    DD_data = json.load(f)

with open('massive_prog_info_incl_vir.json') as f:
    massive_prog = json.load(f)

run_names = list(DD_data.keys())

restart_ds = '0037'

## See if we can match the run_original merger tree until the restart redshift. 

merger_tree_match = {}

for run_name in run_names:
    merger_tree_match[run_name] = {}

potential_ds = {}
matching_ds = {}
for i in run_names:
    potential_ds[i] = []
    matching_ds[i] = []

In [14]:
for i, ds in enumerate(massive_prog):
    match_redshift = DD_data['run_original'][ds]['redshift']
    for run_name in run_names:
        if run_name != 'run_original':
            
            match_position = massive_prog[ds]['position']
            match_mass = massive_prog[ds]['mass']

            target_redshifts = np.array([DD_data[run_name][key]['redshift'] for key in DD_data[run_name].keys()])
            target_outputs = np.array([key for key in DD_data[run_name].keys()])
            
            diff = target_redshifts - match_redshift
            minimum = np.min(np.abs(diff))
            diff_index = np.argwhere(np.abs(diff) == np.min(minimum))[0][0]
            
            target_output = target_outputs[diff_index]
            potential_ds[run_name].append([ds, target_output])

In [15]:
potential_ds

{'run_A': [['0189', '0097'],
  ['0187', '0097'],
  ['0185', '0097'],
  ['0184', '0097'],
  ['0182', '0097'],
  ['0180', '0096'],
  ['0179', '0096'],
  ['0178', '0096'],
  ['0177', '0096'],
  ['0176', '0096'],
  ['0175', '0095'],
  ['0174', '0095'],
  ['0173', '0095'],
  ['0172', '0095'],
  ['0170', '0095'],
  ['0169', '0094'],
  ['0168', '0094'],
  ['0167', '0094'],
  ['0166', '0094'],
  ['0165', '0094'],
  ['0164', '0093'],
  ['0163', '0093'],
  ['0162', '0093'],
  ['0161', '0093'],
  ['0160', '0093'],
  ['0159', '0092'],
  ['0158', '0092'],
  ['0157', '0092'],
  ['0156', '0092'],
  ['0155', '0092'],
  ['0154', '0091'],
  ['0153', '0091'],
  ['0152', '0091'],
  ['0151', '0091'],
  ['0150', '0091'],
  ['0149', '0090'],
  ['0148', '0090'],
  ['0147', '0090'],
  ['0146', '0090'],
  ['0145', '0090'],
  ['0144', '0089'],
  ['0143', '0089'],
  ['0142', '0089'],
  ['0141', '0089'],
  ['0140', '0089'],
  ['0139', '0088'],
  ['0138', '0088'],
  ['0137', '0088'],
  ['0136', '0088'],
  ['0135', 

In [16]:
for i, run_name in enumerate(run_names):
    for j, ds in enumerate(DD_data[run_name]):
        ds_list = []
        for matches in potential_ds[run_name]:
            if ds == matches[1]:
                ds_list.append(matches)
                
        if len(ds_list) == 1:
            matching_ds[run_name].append(ds_list[0])
        
        elif len(ds_list) > 1:
            original_ds = [o[0] for o in ds_list]
            original_redshifts = [DD_data['run_original'][o[0]]['redshift'] for o in ds_list]
            matching_redshift = DD_data[run_name][ds_list[0][1]]['redshift']
            
            diff = np.array(original_redshifts) - matching_redshift
            minimum = np.min(np.abs(diff))
            diff_index = np.argwhere(np.abs(diff) == np.min(minimum))[0][0]
            
            target_output = original_ds[diff_index]
            
            matching_ds[run_name].append([target_output, ds_list[0][1]])
        
        else:
            continue

In [17]:
matching_ds

{'run_A': [['0023', '0023'],
  ['0024', '0024'],
  ['0025', '0025'],
  ['0026', '0026'],
  ['0027', '0027'],
  ['0028', '0028'],
  ['0029', '0029'],
  ['0030', '0030'],
  ['0031', '0031'],
  ['0032', '0032'],
  ['0033', '0033'],
  ['0034', '0034'],
  ['0035', '0035'],
  ['0036', '0036'],
  ['0037', '0037'],
  ['0039', '0038'],
  ['0040', '0039'],
  ['0041', '0040'],
  ['0042', '0041'],
  ['0043', '0042'],
  ['0044', '0043'],
  ['0045', '0044'],
  ['0046', '0045'],
  ['0047', '0046'],
  ['0048', '0047'],
  ['0049', '0048'],
  ['0050', '0049'],
  ['0051', '0050'],
  ['0052', '0051'],
  ['0053', '0052'],
  ['0054', '0053'],
  ['0055', '0054'],
  ['0056', '0055'],
  ['0057', '0056'],
  ['0058', '0057'],
  ['0059', '0058'],
  ['0060', '0059'],
  ['0061', '0060'],
  ['0062', '0061'],
  ['0063', '0062'],
  ['0064', '0063'],
  ['0065', '0064'],
  ['0066', '0065'],
  ['0067', '0066'],
  ['0068', '0067'],
  ['0069', '0068'],
  ['0070', '0069'],
  ['0071', '0070'],
  ['0072', '0071'],
  ['0073', 

In [7]:
matching_halo_info = {}
for key in matching_ds.keys():
    matching_halo_info[key] = {}

In [9]:
for i, run_name in enumerate(matching_ds):
    print(run_name)
    for outputs in matching_ds[run_name]:
        original_output = outputs[0]
        matching_output = outputs[1]
        
        

run_A
run_B
run_C
run_D
run_E
run_fiducial
run_original


In [11]:
matching_ds['run_A']

[['0023', '0023'],
 ['0024', '0024'],
 ['0025', '0025'],
 ['0026', '0026'],
 ['0027', '0027'],
 ['0028', '0028'],
 ['0029', '0029'],
 ['0030', '0030'],
 ['0031', '0031'],
 ['0032', '0032'],
 ['0033', '0033'],
 ['0034', '0034'],
 ['0035', '0035'],
 ['0036', '0036'],
 ['0037', '0037'],
 ['0039', '0038'],
 ['0040', '0039'],
 ['0041', '0040'],
 ['0042', '0041'],
 ['0043', '0042'],
 ['0044', '0043'],
 ['0045', '0044'],
 ['0046', '0045'],
 ['0047', '0046'],
 ['0048', '0047'],
 ['0049', '0048'],
 ['0050', '0049'],
 ['0051', '0050'],
 ['0052', '0051'],
 ['0053', '0052'],
 ['0054', '0053'],
 ['0055', '0054'],
 ['0056', '0055'],
 ['0057', '0056'],
 ['0058', '0057'],
 ['0059', '0058'],
 ['0060', '0059'],
 ['0061', '0060'],
 ['0062', '0061'],
 ['0063', '0062'],
 ['0064', '0063'],
 ['0065', '0064'],
 ['0066', '0065'],
 ['0067', '0066'],
 ['0068', '0067'],
 ['0069', '0068'],
 ['0070', '0069'],
 ['0071', '0070'],
 ['0072', '0071'],
 ['0073', '0072'],
 ['0074', '0073'],
 ['0075', '0074'],
 ['0076', '0

In [8]:
## Path to original run
b = ytree.load('../run_original/rockstar_halos/trees/tree_0_0_0.dat')

for i, run_name in enumerate(matching_ds):
    a = ytree.load(run_dir + run_name + '/rockstar_halos/trees/tree_0_0_0.dat')
    for outputs in matching_ds[run_name]:
        original_output = outputs[0]
        matching_output = outputs[1]

        output_images = run_dir + run_name + '/analysis/halo/'
        
        if int(matching_output) <= int(restart_ds):
            
            matching_redshift = DD_data[run_name][matching_output]['redshift']

            matching_halo_info[run_name][matching_output] = {}

            target_halo = massive_prog[original_output]['halo_id']
            target_mass = massive_prog[original_output]['mass']
            target_position = massive_prog[original_output]['position']
            target_rvir = massive_prog[original_output]['rvir']

            delta = 0.01
            criteria = "(tree['tree', 'redshift'] > " + str(matching_redshift - delta) + ") & (tree['tree', 'redshift'] < " + str(matching_redshift + delta) + ')'

            halos_original = list(b.select_halos(criteria))

            if len(halos_original) == 0:
                delta = 0.05
                criteria = "(tree['tree', 'redshift'] > " + str(matching_redshift - delta) + ") & (tree['tree', 'redshift'] < " + str(matching_redshift + delta) + ')'

                halos_original = list(b.select_halos(criteria))
                if len(halos_original) != 0:
                    print("You've got halos. Let's find the matching halo.")
                else:
                    print('Refine again!')

            else:
                print('Checking to find the matching halo.')

            ## Check to see if we found the matching halo in the original tree:
            original_positions = []
            original_masses = []
            original_ids = []
            for halo in halos_original:
                original_positions.append(halo['position'].to('unitary'))
                original_masses.append(halo['mass'])
                original_ids.append(halo['halo_id'])

            dr_position = np.sqrt(((np.array(original_positions) - target_position)**2).sum(1))
            dr_mass = np.abs(np.array(original_masses) - target_mass)

            min_pos_index = np.argwhere(dr_position == np.min(dr_position))[0][0]
            min_mass_index = np.argwhere(dr_mass == np.min(dr_mass))[0][0]

            if min_pos_index == min_mass_index:
                print('Found the matching halo! Halo ID #' + str(original_ids[min_pos_index]))
                original_redshift = halos_original[min_pos_index]['redshift']
                print(original_redshift)
            else:
                print(original_output)
                print('Cant find matching halo in original tree...')
                print('Must refine criteria.')


            halos = list(a.select_halos("(tree['tree', 'redshift'] == " + str(original_redshift) + ')'))

            if len(halos) == 0:
                halos = list(a.select_halos(criteria))

            positions = []
            masses = []
            ids = []
            rvirs = []
            vir_ratios = []
            for halo in halos:
                positions.append(halo['position'].to('unitary'))
                masses.append(halo['mass'])
                ids.append(halo['halo_id'])
                rvirs.append(halo['virial_radius'].to('unitary'))
                vir_ratios.append(halo['T/|U|'])

            dr_position = np.sqrt(((np.array(positions) - target_position)**2).sum(1))
            dr_mass = np.abs(np.array(masses) - target_mass)

            min_pos_index = np.argwhere(dr_position == np.min(dr_position))[0][0]
            min_mass_index = [i[0] for i in np.argwhere(dr_mass == np.min(dr_mass))]

            if min_pos_index in min_mass_index:
                print('Found the matching halo! Halo ID #' + str(ids[min_pos_index]))
                redshift = halos[min_pos_index]['redshift']

                #matching_halo_info[run_name][matching_output]['halo_id'] = ids[min_pos_index].v.tolist()
                matching_halo_info[run_name][matching_output]['position'] = positions[min_pos_index].v.tolist()
                matching_halo_info[run_name][matching_output]['mass'] = masses[min_pos_index].v.tolist()
                matching_halo_info[run_name][matching_output]['rvir'] = rvirs[min_pos_index].v.tolist()
                matching_halo_info[run_name][matching_output]['T/|U|'] = vir_ratios[min_pos_index]

            else:
                print('Cant find matching halo in original tree...')
                print('Choosing closest halo that is of the same order mass')
                order_mass = int("{:e}".format(target_mass).split('e+')[1])
                mass_criteria  = (np.array(masses) > 10**(order_mass)) & (np.array(masses) < 9*10**(order_mass))

                index = np.argwhere(dr_position == np.min(dr_position))[0][0]

                #matching_halo_info[run_name][matching_output]['halo_id'] = ids[index].v.tolist()
                matching_halo_info[run_name][matching_output]['position'] = positions[index].v.tolist()
                matching_halo_info[run_name][matching_output]['mass'] = masses[index].v.tolist()
                matching_halo_info[run_name][matching_output]['rvir'] = rvirs[index].v.tolist()
                matching_halo_info[run_name][matching_output]['T/|U|'] = vir_ratios[index]

            print('\n')
        else:
            continue

Additional features and improved performance (usually) by saving this arbor with "save_arbor" and reloading:
	>>> a = ytree.load("../run_original/rockstar_halos/trees/tree_0_0_0.dat")
	>>> fn = a.save_arbor()
	>>> a = ytree.load(fn)


Selecting halos (found 8): 100%|███████████████████████████████████████████████████████████████████████████| 530/530 [00:01<00:00, 327.28it/s]


Checking to find the matching halo.
Found the matching halo! Halo ID #3
27.702642


Selecting halos (found 12): 100%|██████████████████████████████████████████████████████████████████████████| 457/457 [00:00<00:00, 753.15it/s]


Found the matching halo! Halo ID #4




...

Checking to find the matching halo.
Found the matching halo! Halo ID #41
21.742779


Selecting halos (found 120): 100%|█████████████████████████████████████████████████████████████████████████| 433/433 [00:00<00:00, 616.64it/s]


Found the matching halo! Halo ID #41




Selecting halos (0 found):   0%|                                                                                      | 0/530 [00:00<?, ?it/s]

KeyboardInterrupt: 